In [0]:
try:
    import langchain
except ImportError:
    %pip install transformers==4.30.2 "unstructured[pdf,docx]==0.10.30" langchain==0.1.5 llama-index==0.9.3 databricks-vectorsearch==0.22 pydantic==1.10.9 mlflow==2.10.1
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.7/886.7 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.1/774.1 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [0]:
%run ./init $reset_all_data=false

USE CATALOG `onlytrends`
using catalog.database `onlytrends`.`onlytrends_db`


DataFrame[]

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS onlytrends.onlytrends_db.volume_reports;

In [0]:
volume_folder =  f"/Volumes/{catalog}/{db}/volume_reports"
url = "https://www.amic.media/media/files/file_352_3520.pdf"
upload_pdf_to_volume(url, volume_folder)
display(dbutils.fs.ls(volume_folder))

saving /Volumes/onlytrends/onlytrends_db/volume_reports/file_352_3520.pdf


path,name,size,modificationTime
dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/checkpoints/,checkpoints/,0,1714933939911
dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/file_352_3520.pdf,file_352_3520.pdf,1216875,1714933940000
dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/hootsuitesocialtrends2024_report_en.pdf,hootsuitesocialtrends2024_report_en.pdf,24950,1714931514000


In [0]:
df = (spark.readStream
        .format('cloudFiles')
        .option('cloudFiles.format', 'BINARYFILE')
        .option("pathGlobFilter", "*.pdf")
        .load('dbfs:'+volume_folder))

# Write the data as a Delta table
(df.writeStream
  .trigger(availableNow=True)
  .option("checkpointLocation", f'dbfs:{volume_folder}/checkpoints/raw_docs')
  .table('pdf_raw').awaitTermination())

In [0]:
%sql SELECT * FROM pdf_raw LIMIT 2

path,modificationTime,length,content
dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/file_352_3520.pdf,2024-05-05T18:32:20Z,1216875,JVBERi0xLjQKJeLjz9MKNDggMCBvYmoKPDwKL0FJUyBmYWxzZQovQk0gL05vcm1hbAovQ0EgMQovT1AgZmFsc2UKL09QTSAxCi9TQSB0cnVlCi9TTWFzayAvTm9uZQovVHlwZSAvRXh0R1N0YXRlCi9jYSAxCi9vcCBmYWxzZQo= (truncated)
dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/hootsuitesocialtrends2024_report_en.pdf,2024-05-05T17:51:54Z,24950,PCFET0NUWVBFIGh0bWw+CjwhLS0KQ29weXJpZ2h0IDIwMTIgTW96aWxsYSBGb3VuZGF0aW9uCgpMaWNlbnNlZCB1bmRlciB0aGUgQXBhY2hlIExpY2Vuc2UsIFZlcnNpb24gMi4wICh0aGUgIkxpY2Vuc2UiKTsKeW91IG1heSA= (truncated)


In [0]:
install_ocr_on_nodes()

OCR libraries installed


In [0]:
from unstructured.partition.auto import partition
import re

def extract_doc_text(x : bytes) -> str:
  # Read files and extract the values with unstructured
  sections = partition(file=io.BytesIO(x))
  def clean_section(txt):
    txt = re.sub(r'\n', '', txt)
    return re.sub(r' ?\.', '.', txt)
  # Default split is by section of document, concatenate them all together because we want to split by sentence instead.
  return "\n".join([clean_section(s.text) for s in sections]) 

In [0]:
import io
import re
with requests.get('https://github.com/databricks-demos/dbdemos-dataset/blob/main/llm/databricks-pdf-documentation/Databricks-Customer-360-ebook-Final.pdf?raw=true') as pdf:
  doc = extract_doc_text(pdf.content)  
  print(doc)

Guide
6 Strategies for Building Personalized Customer Experiences
6 Strategies for Building Personalized Customer Experiences
Contents
Introduction................................................................................................................................................................................................................. 3
1. Building a Foundation for Personalization
Leveraging ML-Based Customer Entity Resolution.............................................................................................................................. 4
2. Estimating Customer Lifetime Value
Building Brand Loyalty With Data................................................................................................................................................................. 6
3. Mitigating Customer Churn
Balancing Acquisition and Retention...........................................................................................................................

In [0]:
from llama_index.langchain_helpers.text_splitter import SentenceSplitter
from llama_index import Document, set_global_tokenizer
from transformers import AutoTokenizer
from typing import Iterator
from pyspark.sql.functions import pandas_udf
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf, length, pandas_udf
import os
import mlflow
from typing import Iterator
from mlflow import MlflowClient


# Reduce the arrow batch size as our PDF can be big in memory
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 10)

@pandas_udf("array<string>")
def read_as_chunk(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    #set llama2 as tokenizer to match our model size (will stay below BGE 1024 limit)
    set_global_tokenizer(
      AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
    )
    #Sentence splitter from llama_index to split on sentences
    splitter = SentenceSplitter(chunk_size=500, chunk_overlap=50)
    def extract_and_split(b):
      txt = extract_doc_text(b)
      nodes = splitter.get_nodes_from_documents([Document(text=txt)])
      return [n.text for n in nodes]

    for x in batch_iter:
        yield x.apply(extract_and_split)

In [0]:
from mlflow.deployments import get_deploy_client

# bge-large-en Foundation models are available using the /serving-endpoints/databricks-bge-large-en/invocations api. 
deploy_client = get_deploy_client("databricks")
 
## NOTE: if you change your embedding model here, make sure you change it in the query step too
embeddings = deploy_client.predict(endpoint="databricks-bge-large-en", inputs={"input": ["What is Apache Spark?"]})
pprint(embeddings)

{'data': [{'embedding': [0.0185699462890625, -0.01403045654296875, -0.057647705078125,
                         0.003448486328125, 0.008575439453125, -0.0216827392578125,
                         -0.0247344970703125, -0.0047149658203125, 0.0136260986328125,
                         0.050323486328125, -0.027496337890625, -0.0147247314453125,
                         0.05474853515625, -0.053802490234375, -0.01025390625, -0.0161895751953125,
                         -0.018768310546875, -0.017181396484375, -0.051177978515625,
                         0.0178680419921875, 0.0042877197265625, 0.028533935546875,
                         -0.05548095703125, -0.037750244140625, -0.0012273788452148438,
                         0.0201873779296875, -0.0467529296875, 0.015869140625, 0.09375,
                         0.0194854736328125, -0.044708251953125, -0.01235198974609375,
                         -0.00634765625, -0.0291900634765625, 0.04327392578125, -0.02532958984375,
                         0

In [0]:
%sql
--Note that we need to enable Change Data Feed on the table to create the index
CREATE TABLE IF NOT EXISTS pdf_reports (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY,
  url STRING,
  content STRING,
  embedding ARRAY <FLOAT>
) TBLPROPERTIES (delta.enableChangeDataFeed = true); 

In [0]:
@pandas_udf("array<float>")
def get_embedding(contents: pd.Series) -> pd.Series:
    import mlflow.deployments
    deploy_client = mlflow.deployments.get_deploy_client("databricks")
    def get_embeddings(batch):
        #Note: this will fail if an exception is thrown during embedding creation (add try/except if needed) 
        response = deploy_client.predict(endpoint="databricks-bge-large-en", inputs={"input": batch})
        return [e['embedding'] for e in response.data]

    # Splitting the contents into batches of 150 items each, since the embedding model takes at most 150 inputs per request.
    max_batch_size = 150
    batches = [contents.iloc[i:i + max_batch_size] for i in range(0, len(contents), max_batch_size)]

    # Process each batch and collect the results
    all_embeddings = []
    for batch in batches:
        all_embeddings += get_embeddings(batch.tolist())

    return pd.Series(all_embeddings)

In [0]:
(spark.readStream.table('pdf_raw')
      .withColumn("content", F.explode(read_as_chunk("content")))
      .withColumn("embedding", get_embedding("content"))
      .selectExpr('path as url', 'content', 'embedding')
  .writeStream
    .trigger(availableNow=True)
    .option("checkpointLocation", f'dbfs:{volume_folder}/checkpoints/pdf_chunks')
    .table('pdf_reports').awaitTermination())

In [0]:
%sql
SELECT * FROM pdf_reports WHERE url like '%.pdf' limit 10

id url content embedding 2 dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/file_352_3520.pdf On behalf of our talented marketing and research teams, I’m pleased to share
Hootsuite’s Social Media Trends 2022 Report.
This year’s report analyzes the topics that marketers consistently want to learn
more about: emerging industry behaviors, breakthrough platforms, and new
normals. In this year’s report we also wanted to look at the bigger picture—the
themes, innovations and infrastructure that have become fixtures in the social
space, regardless of yearly trends.
After the turbulent times we’ve all just experienced, when many brands and
companies were forced into operating tactically just to survive, it’s time for
marketers to start thinking strategically again—considering how to create
connected, enduring online communities; championing the infinite potential
of social commerce; and committing to the possibilities that elevated customer
care creates.
Use the data and be courageous! We want to support you in making bold choices
designed for growth—choices informed by research, data, vision and purpose.
Enjoy the report and let’s make them wonder how you did it.
Tom Keiser CEO, Hootsuite
Social Trends 2022
Dive in! List(0.027786255, 0.041046143, -0.039123535, 0.047576904, -0.0033035278, -0.021697998, -0.0059890747, 0.03555298, 0.08868408, 0.0027503967, -0.03125, 0.0030651093, 0.004676819, 0.012107849, -0.004940033, -0.016326904, 0.004043579, -0.033447266, -0.021972656, 0.020431519, 0.0011816025, 0.048431396, -0.056488037, -0.058288574, -0.036712646, 0.057800293, -0.009262085, -0.009277344, 0.09197998, 0.038360596, 0.022979736, -0.017471313, -0.031555176, -0.038604736, -0.06549072, -0.011314392, 0.030563354, -0.023468018, -0.017120361, -0.03677368, -0.00818634, 0.03062439, 0.081848145, -0.051239014, -0.024337769, -0.007045746, 0.00233078, -0.017822266, -0.019622803, -0.027923584, 0.028671265, -0.0032978058, -0.017669678, -0.015045166, 0.018661499, -0.028625488, -0.016082764, 0.017990112, 0.010284424, 0.053649902, 0.016540527, 0.036987305, 0.03881836, -0.040283203, 0.005947113, -0.00819397, 0.0058288574, -0.034973145, 0.00894165, -0.024475098, -0.041412354, 8.0633163E-4, -0.061920166, 0.02104187, -0.020584106, -0.011131287, -0.015853882, -0.035064697, -0.04586792, -0.02053833, -0.0019569397, 0.024627686, 0.01197052, 0.020767212, -0.030090332, 0.022964478, 0.038970947, -0.0034255981, 0.040496826, 0.007534027, 0.039611816, 0.03527832, -0.02128601, -0.008728027, 0.051757812, 0.053710938, 0.006542206, 0.014533997, -0.017929077, 0.018829346, 0.018676758, 0.03744507, -0.009307861, 0.049224854, -0.057281494, 0.042816162, 0.0637207, -0.05569458, 0.035614014, -0.07171631, -5.376339E-5, -0.011001587, 0.010482788, -0.015426636, -0.0035972595, 0.061065674, -0.016845703, 0.011375427, -0.035888672, -0.011299133, -0.027374268, 0.020736694, -0.017837524, -0.00440979, -0.021728516, 0.0071105957, 0.035491943, 0.036590576, -0.042419434, 0.016845703, -0.03640747, -0.03540039, 0.011688232, 0.06359863, 0.017410278, -2.7108192E-4, 0.020904541, 0.015258789, 0.019500732, -0.037963867, -0.006916046, 0.021835327, 0.012466431, 0.07952881, -0.016403198, 0.040039062, -0.055511475, -0.055389404, -0.027694702, 0.0034828186, 0.028778076, 0.08337402, -0.027786255, -0.0044898987, -0.030166626, -0.03781128, -0.02734375, -0.0033397675, -0.021621704, 0.012535095, -0.019927979, 0.058410645, -0.028335571, 0.026473999, -0.0770874, 0.035339355, 0.024719238, -0.011077881, -0.02609253, -0.0054626465, 0.043823242, 0.039794922, -0.02619934, 0.0012865067, 0.01486969, 0.05404663, -0.0047340393, -0.022415161, 0.014305115, 0.015472412, -0.011161804, -0.0073928833, 0.0052986145, 0.044433594, -0.028137207, 0.016281128, 0.029022217, -0.05319214, -0.023590088, -0.06939697, -0.008857727, 0.013900757, -0.046569824, -0.002855301, -0.010643005, 0.002571106, -0.020004272, -0.0011377335, 0.034179688, -0.03555298, -0.046447754, 0.0141067505, -0.053771973, 0.02772522, -0.016357422, 0.002044

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME)
print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
Waiting for endpoint to be ready, this can take a few min... {'name': 'onlytrends_vs_endpoint', 'creator': 'saumo@usesneakpeek.com', 'creation_timestamp': 1714933085247, 'last_updated_timestamp': 1714933085247, 'endpoint_type': 'STANDARD', 'last_updated_user': 'saumo@usesneakpeek.com', 'id': 'dd99e783-06a4-42eb-9cfe-caf5ec099da3', 'endpoint_status': {'state': 'PROVISIONING'}, 'num_indexes': 0}
Waiting for endpoint to be ready, this can take a few min... {'name': 'onlytrends_vs_endpoint', 'creator': 'saumo@usesneakpeek.com', 'creation_timestamp': 1714933085247, 'last_updated_timestamp': 1714933085247, 'endpoint_type': 'STANDARD', 'last_updated_user': 'saumo@usesneakpeek.com', 'id': 'dd99e783-06a4-42eb-9cfe-caf5ec099da3', 'endpoint_status': {'state': 'PROVIS

In [0]:
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c

#The table we'd like to index
source_table_fullname = f"{catalog}.{db}.pdf_reports"
# Where we want to store our index
vs_index_fullname = f"{catalog}.{db}.pdf_reports_self_managed_vs_index"

if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  vsc.create_delta_sync_index(
    endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
    index_name=vs_index_fullname,
    source_table_name=source_table_fullname,
    pipeline_type="TRIGGERED", #Sync needs to be manually triggered
    primary_key="id",
    embedding_dimension=1024, #Match your model embedding size (bge)
    embedding_vector_column="embedding"
  )
  #Let's wait for the index to be ready and all our embeddings to be created and indexed
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
else:
  #Trigger a sync to update our vs content with the new data saved in the table
  wait_for_index_to_be_ready(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
  vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).sync()

In [0]:
question = "What are some brand strategy trends?"

response = deploy_client.predict(endpoint="databricks-bge-large-en", inputs={"input": [question]})
embeddings = [e['embedding'] for e in response.data]

results = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname).similarity_search(
  query_vector=embeddings[0],
  columns=["url", "content"],
  filters= {"url LIKE": "file_352_3520"},
  num_results=1)
docs = results.get('result', {}).get('data_array', [])
pprint(docs)

[['dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/file_352_3520.pdf',
  'Enjoy the report and let’s make them wonder how you did it.\n'
  'Tom Keiser CEO, Hootsuite\n'
  'Social Trends 2022\n'
  'Dive in!\n'
  '4\n'
  '5\n'
  '6\n'
  '13\n'
  '20\n'
  '26\n'
  '33\n'
  '41\n'
  'Executive summary\n'
  'Methodology\n'
  'The Brand Strategy Trend\n'
  'Brands\tfinally\tget\tcommunity\tright\t(with\tthe\thelp\tof\tcreators)\n'
  'The Social Advertising Trend\n'
  'Marketers\tget\tcreative\tas\tconsumers\twise\tup\tto\tsocial\tads\n'
  'The ROI Trend \t Social\tquietly\tmatures\tout\tof\tthe\tmarketing\tdepartment\n'
  'The Social Commerce Trend\n'
  'Social\tbecomes\tthe\theart\tof\tthe\tpost-pandemic\tshopping\texperience\n'
  'The Customer Care Trend\n'
  'Social\tmarketers\tsave\ttheir\tbrands\tfrom\tthe\tcustomer\tservice\tapocalypse\n'
  'Full survey results\n'
  'Executive summary\n'
  'Based on our annual survey of 18,100 marketers, exhaustive research, and interviews with '

In [0]:
%pip install mlflow==2.10.1 lxml==4.9.3 langchain==0.1.5 databricks-vectorsearch==0.22 cloudpickle==2.2.1 databricks-sdk==0.18.0 cloudpickle==2.2.1 pydantic==2.5.2
%pip install pip mlflow[databricks]==2.10.1

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.1
    Not uninstalling lxml at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-d4fd89d1-f47f-4c6c-b99c-00f819f538dd
    Can't uninstall 'lxml'. No files were found to uninstall.
  Attempting uninst

In [0]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatDatabricks
from langchain.schema.output_parser import StrOutputParser

prompt = PromptTemplate(
  input_variables = ["question"],
  template = "You are an assistant. Give a short answer to this question: {question}"
)
chat_model = ChatDatabricks(endpoint="databricks-dbrx-instruct", max_tokens = 500)

chain = (
  prompt
  | chat_model
  | StrOutputParser()
)
print(chain.invoke({"question": "What is Spark?"}))

Spark is an open-source, distributed computing system used for big data processing and analytics. It provides an interface for programming entire clusters with implicit data parallelism and fault tolerance. Spark can run on Hadoop, Apache Mesos, Kubernetes, standalone, or in the cloud, and is able to access diverse data sources including HDFS, Cassandra, HBase, and S3. It offers over 80 high-level operators for interactive queries, streaming, machine learning, and graph processing.


In [0]:
prompt_with_history_str = """
Your are a Trends chatbot. Please answer Trends question only. If you don't know or not related to Trends, don't answer.

Here is a history between you and a human: {chat_history}

Now, please answer this question: {question}
"""

prompt_with_history = PromptTemplate(
  input_variables = ["chat_history", "question"],
  template = prompt_with_history_str
)

In [0]:
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

#The question is the last entry of the history
def extract_question(input):
    return input[-1]["content"]

#The history is everything before the last question
def extract_history(input):
    return input[:-1]

chat_model = ChatDatabricks(endpoint="databricks-dbrx-instruct", max_tokens = 200)

is_question_about_databricks_str = """
You are classifying documents to know if this question is related with Databricks in AWS, Azure and GCP, Workspaces, Databricks account and cloud infrastructure setup, Data Science, Data Engineering, Big Data, Datawarehousing, SQL, Python and Scala or something from a very different field. Also answer no if the last part is inappropriate. 

Here are some examples:

Question: Knowing this followup history: What is Databricks?, classify this question: Do you have more details?
Expected Response: Yes

Question: Knowing this followup history: What is Databricks?, classify this question: Write me a song.
Expected Response: No

Only answer with "yes" or "no". 

Knowing this followup history: {chat_history}, classify this question: {question}
"""

is_question_about_databricks_prompt = PromptTemplate(
  input_variables= ["chat_history", "question"],
  template = is_question_about_databricks_str
)

is_about_databricks_chain = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | is_question_about_databricks_prompt
    | chat_model
    | StrOutputParser()
)

#Returns "Yes" as this is about Databricks: 
print(is_about_databricks_chain.invoke({
    "messages": [
        {"role": "user", "content": "What is Apache Spark?"}, 
        {"role": "assistant", "content": "Apache Spark is an open-source data processing engine that is widely used in big data analytics."}, 
        {"role": "user", "content": "Does it support streaming?"}
    ]
}))

Yes.


In [0]:
index_name=f"{catalog}.{db}.pdf_reports_self_managed_vs_index"
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
print(host)

#Let's make sure the secret is properly setup and can access our vector search index. Check the quick-start demo for more guidance
test_demo_permissions(host, secret_scope="dbdemos", secret_key="rag_sp_token", vs_endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME, index_name=index_name, embedding_endpoint_name="databricks-bge-large-en", managed_embeddings = False)

https://dbc-a2aec960-4659.cloud.databricks.com


Configuration error: Unkown error accessing the Vector Search index onlytrends.onlytrends_db.pdf_reports_self_managed_vs_index using the endpoint onlytrends_vs_endpoint and secret dbdemos/rag_sp_token 
 
 Why are we getting this error? 
 The model is using the Pat Token saved with the secret dbdemos/rag_sp_token to access your vector search index 'onlytrends.onlytrends_db.pdf_reports_self_managed_vs_index' (host:https://dbc-a2aec960-4659.cloud.databricks.com). 
 To do so, the principal owning the Pat Token must have USAGE permission on your schema and READ permission on the index. 
 The principal is the one who generated the token you saved as secret: `saumo@usesneakpeek.com`. 
 Note: Production-grade deployement should to use a Service Principal ID instead. 
 
 Here is how you can fix it: 
 Make sure your Service Principal has USE privileve on the schema :
 
 spark.sql('GRANT USAGE ON CATALOG `onlytrends` TO `saumo@usesneakpeek.com`'); 
 spark.sql('GRANT USAGE ON DATABASE `onlytrends`.`onlytrends_db` TO `saumo@usesneakpeek.com`'); 
 
 
 Grant SELECT access to your SP to your index: 
 
 from databricks.sdk import WorkspaceClient 
 import databricks.sdk.service.catalog as c 
 WorkspaceClient().grants.update(c.SecurableType.TABLE, "onlytrends.onlytrends_db.pdf_reports_self_managed_vs_index", 
 changes=[c.PermissionsChange(add=[c.Privilege["SELECT"]], principal="saumo@usesneakpeek.com")])
 
 
 If this is still not working, make sure the value saved in your dbdemos/rag_sp_token secret is your SP pat token . 
 Note: if you're using a shared demo workspace, please do not change the secret value if was set to a valid SP value by your admins. 

 
 
 Detailed error trying to access the endpoint:
 No module named 'databricks.vector_search'

In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain_community.vectorstores import DatabricksVectorSearch
from langchain_community.embeddings import DatabricksEmbeddings
from langchain.chains import RetrievalQA

os.environ['DATABRICKS_TOKEN'] = dbutils.secrets.get("dbdemos", "rag_sp_token")

embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

def get_retriever(persist_dir: str = None):
    os.environ["DATABRICKS_HOST"] = host
    #Get the vector search index
    vsc = VectorSearchClient(workspace_url=host, personal_access_token=os.environ["DATABRICKS_TOKEN"])
    vs_index = vsc.get_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        index_name=index_name
    )

    # Create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index, text_column="content", embedding=embedding_model, columns=["url"]
    )
    return vectorstore.as_retriever(search_kwargs={'k': 4, "filters": {"url LIKE": "file_352_3520"},})

retriever = get_retriever()

retrieve_document_chain = (
    itemgetter("messages") 
    | RunnableLambda(extract_question)
    | retriever
)
print(retrieve_document_chain.invoke({"messages": [{"role": "user", "content": "What is Apache Spark?"}]}))

[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
[Document(page_content='Instead of creating communities\nfrom the ground up, they engage\nwith communities that form\norganically. They shift from\ntop-down communication to\nmeaningful conversations. They\nbuild relationships that evolve,\ngrow and stand the test of time.”\nSid Lee The Belong Effect\n10\nT h e B r a n d S\nt r a t e g y\nT r e n d\nStorage solution company BiGDUG dives into TikTok with the help of @theP001guy\nIf you think commercial shelving,\nThey’ve since become a regular fixture on Laflin’s feed, even showing up as\nracking, and storage providers aren’t\nrecurring characters in his videos.\nlikely candidates for TikTok brand\nfame, UK-based BiGDUG is here to\nprove you wrong. While people were\nin lockdown, BiGDUG saw a boom in\

In [0]:
from langchain.schema.runnable import RunnableBranch

generate_query_to_retrieve_context_template = """
Based on the chat history below, we want you to generate a query for an external data source to retrieve relevant documents so that we can better answer the question. The query should be in natual language. The external data source uses similarity search to search for relevant documents in a vector space. So the query should be similar to the relevant documents semantically. Answer with only the query. Do not add explanation.

Chat history: {chat_history}

Question: {question}
"""

generate_query_to_retrieve_context_prompt = PromptTemplate(
  input_variables= ["chat_history", "question"],
  template = generate_query_to_retrieve_context_template
)

generate_query_to_retrieve_context_chain = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | RunnableBranch(  #Augment query only when there is a chat history
      (lambda x: x["chat_history"], generate_query_to_retrieve_context_prompt | chat_model | StrOutputParser()),
      (lambda x: not x["chat_history"], RunnableLambda(lambda x: x["question"])),
      RunnableLambda(lambda x: x["question"])
    )
)

#Let's try it
output = generate_query_to_retrieve_context_chain.invoke({
    "messages": [
        {"role": "user", "content": "What is Apache Spark?"}
    ]
})
print(f"Test retriever query without history: {output}")

output = generate_query_to_retrieve_context_chain.invoke({
    "messages": [
        {"role": "user", "content": "What is Apache Spark?"}, 
        {"role": "assistant", "content": "Apache Spark is an open-source data processing engine that is widely used in big data analytics."}, 
        {"role": "user", "content": "Does it support streaming?"}
    ]
})
print(f"Test retriever question, summarized with history: {output}")

Test retriever query without history: What is Apache Spark?
Test retriever question, summarized with history: "Exploring Apache Spark's streaming capabilities and applications in real-time data processing"


In [0]:
from langchain.schema.runnable import RunnableBranch, RunnableParallel, RunnablePassthrough

question_with_history_and_context_str = """
You are a trustful assistant for Databricks users. You are answering python, coding, SQL, data engineering, spark, data science, AI, ML, Datawarehouse, platform, API or infrastructure, Cloud administration question related to Databricks. If you do not know the answer to a question, you truthfully say you do not know. Read the discussion to get the context of the previous conversation. In the chat discussion, you are referred to as "system". The user is referred to as "user".

Discussion: {chat_history}

Here's some context which might or might not help you answer: {context}

Answer straight, do not repeat the question, do not start with something like: the answer to the question, do not add "AI" in front of your answer, do not say: here is the answer, do not mention the context or the question.

Based on this history and context, answer this question: {question}
"""

question_with_history_and_context_prompt = PromptTemplate(
  input_variables= ["chat_history", "context", "question"],
  template = question_with_history_and_context_str
)

def format_context(docs):
    return "\n\n".join([d.page_content for d in docs])

def extract_source_urls(docs):
    return [d.metadata["url"] for d in docs]

relevant_question_chain = (
  RunnablePassthrough() |
  {
    "relevant_docs": generate_query_to_retrieve_context_prompt | chat_model | StrOutputParser() | retriever,
    "chat_history": itemgetter("chat_history"), 
    "question": itemgetter("question")
  }
  |
  {
    "context": itemgetter("relevant_docs") | RunnableLambda(format_context),
    "sources": itemgetter("relevant_docs") | RunnableLambda(extract_source_urls),
    "chat_history": itemgetter("chat_history"), 
    "question": itemgetter("question")
  }
  |
  {
    "prompt": question_with_history_and_context_prompt,
    "sources": itemgetter("sources")
  }
  |
  {
    "result": itemgetter("prompt") | chat_model | StrOutputParser(),
    "sources": itemgetter("sources")
  }
)

irrelevant_question_chain = (
  RunnableLambda(lambda x: {"result": 'I cannot answer questions that are not about Databricks.', "sources": []})
)

branch_node = RunnableBranch(
  (lambda x: "yes" in x["question_is_relevant"].lower(), relevant_question_chain),
  (lambda x: "no" in x["question_is_relevant"].lower(), irrelevant_question_chain),
  irrelevant_question_chain
)

full_chain = (
  {
    "question_is_relevant": is_about_databricks_chain,
    "question": itemgetter("messages") | RunnableLambda(extract_question),
    "chat_history": itemgetter("messages") | RunnableLambda(extract_history),    
  }
  | branch_node
)

In [0]:
import json
non_relevant_dialog = {
    "messages": [
        {"role": "user", "content": "What is Apache Spark?"}, 
        {"role": "assistant", "content": "Apache Spark is an open-source data processing engine that is widely used in big data analytics."}, 
        {"role": "user", "content": "Why is the sky blue?"}
    ]
}
print(f'Testing with a non relevant question...')
response = full_chain.invoke(non_relevant_dialog)
display_chat(non_relevant_dialog["messages"], response)

Testing with a non relevant question...


What is Apache Spark?
 
 
 
 Apache Spark is an open-source data processing engine that is widely used in big data analytics.
 
 
 Why is the sky blue?
 
 
 
 I cannot answer questions that are not about Databricks.

In [0]:
dialog = {
    "messages": [
        {"role": "user", "content": "What is Apache Spark?"}, 
        {"role": "assistant", "content": "Apache Spark is an open-source data processing engine that is widely used in big data analytics."}, 
        {"role": "user", "content": "Does it support streaming?"}
    ]
}
print(f'Testing with relevant history and question...')
response = full_chain.invoke(dialog)
display_chat(dialog["messages"], response)

Testing with relevant history and question...


What is Apache Spark?
 
 
 
 Apache Spark is an open-source data processing engine that is widely used in big data analytics.
 
 
 Does it support streaming?
 
 
 
 Yes, Apache Spark supports streaming. It can process real-time data through Spark Streaming, a component that enables scalable, high-throughput, fault-tolerant stream processing of live data streams. Sources: dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/file_352_3520.pdf 
 dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/file_352_3520.pdf 
 dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/file_352_3520.pdf 
 dbfs:/Volumes/onlytrends/onlytrends_db/volume_reports/file_352_3520.pdf